### Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

### Dataset

In [2]:
real_estate_data = pd.read_csv("../data/Real_estate.csv")
print (real_estate_data.shape)
real_estate_data.head()

(414, 8)


,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [3]:
414/12

34.5

### Spltting data into train and test sets and normalizing it

In [4]:
# Train and Test
X = real_estate_data.iloc[:, 1:7]
y = real_estate_data.iloc[:, -1]

# Normalize the features to the [0, 1] range using min max
x_max = X.max(axis=0)
x_min = X.min(axis=0)
X_normalized = (X - x_min) / (x_max - x_min) 

# Transforming train dataset into to tensors
x_tensor = torch.tensor(X_normalized.values, dtype=torch.float)
y_tensor = torch.tensor(y.values, dtype=torch.float)

# Dataset and DataLoader for mini-batch processing
dataset = TensorDataset(x_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size = 30, shuffle=True)

### Neural Network Structure 

In [5]:
# Neural Network structure
nn_model = nn.Sequential(
    nn.Linear(6, 30),
    nn.ReLU(),
    nn.Linear(30,30),
    nn.ReLU(),
    nn.Linear(30,30),
    nn.ReLU(),
    nn.Linear(30, 1)
)

criterion = nn.MSELoss()
optimizer = optim.SGD(nn_model.parameters(), lr = 0.001)

# Number of epochs
num_epochs = 100

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        
        # Forward pass
        outputs = nn_model(inputs)
        
        # Match outputs shape
        targets = targets.view(-1, 1)
        
        # Compute Loss
        loss = criterion(outputs, targets)
        
        # Backward pass and optimize
        optimizer.zero_grad() # Clear existent gradients
        loss.backward() # compute gradients
        optimizer.step() # Update parameters
        
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 37.5023
Epoch [20/100], Loss: 99.7048
Epoch [30/100], Loss: 73.1872
Epoch [40/100], Loss: 28.5178
Epoch [50/100], Loss: 108.7546
Epoch [60/100], Loss: 75.8795
Epoch [70/100], Loss: 31.1016
Epoch [80/100], Loss: 204.1349
Epoch [90/100], Loss: 120.6146
Epoch [100/100], Loss: 19.7027


### MLP Model Evaluation

In [6]:
y_pred = nn_model(x_tensor)

y_pred = y_pred.detach().numpy()
y_pred = y_pred[:, 0]

# Calculate R-squared using final predictions
SSR = ((y - y_pred) ** 2).sum()
SST = ((y - y.mean()) ** 2).sum()
r2 = round(1 - (SSR / SST), )
print("R^2: ", r2)

R^2:  0.649
